In [1]:
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 23.4 MB/s eta 0:00:00


In [2]:
import numpy as np
import random
from collections import Counter

In [3]:
from Bio import SeqIO

In [4]:
random.seed(44)

#Симуляция секвенирования (3 балла)

*   Первое задание заключается в том, чтобы написать симуляцию секвенирования ридов при помощи illumina. Для этого необходимо сгенерировать случайно строку длинной 50 000 символов, над алфавитом {A, T, G, C}.
*   После этого случайно выбирать подстроки, длинна которых распределена нормально со средним 250 и среднеквадратическим отклонением 30, каждую такую подстроку "секвенировать".
*   Под словом "секвенировать" в данном случае имеется ввиду симуляция того процесса, который мы обсуждали на лекции. Вы считываете очередной символ подстроки, например, 100 раз, но в N случаях из 100 ошибаетесь и считываете любой нуклеотид, кроме правильного с одинаковой вероятностью. N принадлежит равномерному распределению от 0 до 60.
*   По получившемуся набору вычисляете наибоее вероятный нуклеотид (тот, которого больше всего) и его качество прочтения, чтобы записать его в формате FASTQ. При симуляции важно запоминать ID рида и для каждого ID позиции, в которых нуклеотид был считан неверно и какой должен быть на самом деле (для этого просто можно хранить 2 числа и помнить что у вас есть исходная строка, откуда берутся риды). Это понадобится для выполнения следующих заданий.
Всего ридов пусть будет 50К.

In [5]:
# Генерация случайной строки длиной 50,000
nucleotides = ['A', 'T', 'G', 'C']
sequence_length = 50000
sequence = ''.join(random.choices(nucleotides, k=sequence_length))

In [17]:
# Функция для генерации рида с ошибками
def generate_read(sequence, start_pos, read_length):
    read = sequence[start_pos:start_pos + read_length]
    sequenced_read = []
    qualities = []
    error_positions = []
    for i, base in enumerate(read):
        base_counts = Counter()
        N = random.randint(0, 75)
        for iter in range(100):
            if iter < N:
                base_counts[random.choice([n for n in nucleotides if n != base])] += 1
            else:
                base_counts[base] += 1
        most_common_base, count = base_counts.most_common(1)[0]
        sequenced_read.append(most_common_base)


        P = 1 - (count / 100)
        if P < 5 * 10**(-10):
            quality_char = chr(126)
        elif P == 0:
            quality_char = chr(33)
        else:
            Q = -10 * np.log10(P)
            quality_char = chr(int(Q) + 33)
        qualities.append(quality_char)
        # print(P, N, quality_char)
        if most_common_base != base:
            error_positions.append((start_pos, i))

    return ''.join(sequenced_read), ''.join(qualities), error_positions

In [18]:
num_reads = 50000
read_lengths = np.random.normal(250, 30, num_reads).astype(int)
fastq_data = []
error_info = {}
for read_id in range(num_reads):
    start_pos = random.randint(0, sequence_length - read_lengths[read_id])
    read_length = read_lengths[read_id]
    read, qualitie, errors = generate_read(sequence, start_pos, read_length)

    fastq_entry = f"@read_{read_id}\n{read}\n+\n{qualitie}\n"
    fastq_data.append(fastq_entry)

    if errors:
        error_info[read_id] = errors

In [19]:
# Запись в файл FASTQ
with open('sequenced_reads.fastq', 'w') as fastq_file:
    fastq_file.writelines(fastq_data)

# Удаление ошибок (Trimmomatic) (2 балла)

Получившиеся риды обработайте Trimmomatic со стандартными параметрами с лекции. Пользуясь тем, что вы знаете, для каждого рида ground truth, посчитайте, в скольких случаях Trimmomatic удалил нуклеотиды, которые были считаны верно, а в скольких действительно удалил ошибочные.

##Загрузка

In [ ]:
!apt-get install trimmomatic

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
trimmomatic is already the newest version (0.39+dfsg-2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [ ]:
!conda install -c bioconda trimmomatic

Channels:
 - bioconda
 - conda-forge
Platform: linux-64
Solving environment: \ | / - done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.5.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



In [10]:
!wget https://www.bioinformatics.babraham.ac.uk/projects/fastqc/fastqc_v0.11.9.zip
!unzip fastqc_v0.11.9.zip

--2024-06-04 20:27:55--  https://www.bioinformatics.babraham.ac.uk/projects/fastqc/fastqc_v0.11.9.zip
Resolving www.bioinformatics.babraham.ac.uk (www.bioinformatics.babraham.ac.uk)... 149.155.133.4
Connecting to www.bioinformatics.babraham.ac.uk (www.bioinformatics.babraham.ac.uk)|149.155.133.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10249221 (9.8M) [application/zip]
Saving to: ‘fastqc_v0.11.9.zip’

fastqc_v0.11.9.zip  100%[===================>]   9.77M  10.2MB/s    in 1.0s    

2024-06-04 20:27:57 (10.2 MB/s) - ‘fastqc_v0.11.9.zip’ saved [10249221/10249221]

Archive:  fastqc_v0.11.9.zip
  inflating: FastQC/cisd-jhdf5.jar   
   creating: FastQC/Configuration/
  inflating: FastQC/Configuration/adapter_list.txt  
  inflating: FastQC/Configuration/contaminant_list.txt  
  inflating: FastQC/Configuration/limits.txt  
  inflating: FastQC/fastqc           
  inflating: FastQC/fastqc_icon.ico  
   creating: FastQC/Help/
   creating: FastQC/Help/1 Introductio

##trimming

*   Удаление адаптеров (ILLUMINACLIP:TruSeq3-SE:2:30:10) у нас их нету
*   Удаление низкокачественных вначале (с качеством хуже 3) (LEADING:3)
*   Удаление низкокачественных в конце (с качеством хуже 3) (TRAILING:3)
*   Сканировать окном в 4 нуклеотида, если среднее качество в окне ниже 15, то удалять(SLIDINGWINDOW:4:15) опустим среднее качество чтобы не выкидывало все риды.
*   Удалять риды короче 36 нуклеотидов (MINLEN:36)
*   SE - не парные чтения




java -jar /home/sveta/soft/Trimmomatic-0.39/trimmomatic-0.39.jar SE /home/sveta/Загрузки/sequenced_reads_1.fastq /home/sveta/Загрузки/trimmed_reads.fastq SLIDINGWINDOW:4:2 LEADING:3 TRAILING:3 MINLEN:36
TrimmomaticSE: Started with arguments:
 /home/sveta/Загрузки/sequenced_reads_1.fastq /home/sveta/Загрузки/trimmed_reads.fastq SLIDINGWINDOW:4:2 LEADING:3 TRAILING:3 MINLEN:36
Automatically using 4 threads
Quality encoding detected as phred33
Input Reads: 50000 Surviving: 29213 (58,43%) Dropped: 20787 (41,57%)
TrimmomaticSE: Completed successfully


In [25]:
def analyze_trimmed_reads(trimmed_data, error_data):
    correct_deletions = 0
    incorrect_deletions = 0

    trimmed_reads = list(SeqIO.parse('trimmed_reads.fastq', 'fastq'))

    for record in trimmed_reads:
        read_id = int(record.id[5:])
        original_errors = error_data[read_id]
        trimmed_sequence = str(record.seq)

        for start, pos in original_errors:
            if pos < len(trimmed_sequence):
                correct_base = sequence[pos+start]
                if trimmed_sequence[pos] != correct_base:
                    incorrect_deletions += 1
                else:
                    correct_deletions += 1

    return correct_deletions, incorrect_deletions

In [26]:
# Чтение обработанных ридов
with open('trimmed_reads.fastq', 'r') as f:
    trimmed_data = f.readlines()

correct_deletions, incorrect_deletions = analyze_trimmed_reads(trimmed_data, error_info)
print(f"Correct deletions: {correct_deletions}")
print(f"Incorrect deletions: {incorrect_deletions}")

Correct deletions: 10506
Incorrect deletions: 121511


Correct deletions: 10506
Incorrect deletions: 121511

## Можно посмотреть отчеты FastQC

In [11]:
!chmod +x FastQC/fastqc

In [45]:
!FastQC/fastqc  sequenced_reads.fastq

Started analysis of sequenced_reads.fastq
Approx 5% complete for sequenced_reads.fastq
Approx 10% complete for sequenced_reads.fastq
Approx 15% complete for sequenced_reads.fastq
Approx 20% complete for sequenced_reads.fastq
Approx 25% complete for sequenced_reads.fastq
Approx 30% complete for sequenced_reads.fastq
Approx 35% complete for sequenced_reads.fastq
Approx 40% complete for sequenced_reads.fastq
Approx 45% complete for sequenced_reads.fastq
Approx 50% complete for sequenced_reads.fastq
Approx 55% complete for sequenced_reads.fastq
Approx 60% complete for sequenced_reads.fastq
Approx 65% complete for sequenced_reads.fastq
Approx 70% complete for sequenced_reads.fastq
Approx 75% complete for sequenced_reads.fastq
Approx 80% complete for sequenced_reads.fastq
Approx 85% complete for sequenced_reads.fastq
Approx 90% complete for sequenced_reads.fastq
Approx 95% complete for sequenced_reads.fastq
Approx 100% complete for sequenced_reads.fastq
Analysis complete for sequenced_reads.

In [46]:
!FastQC/fastqc  trimmed_reads.fastq

Started analysis of trimmed_reads.fastq
Approx 5% complete for trimmed_reads.fastq
Approx 10% complete for trimmed_reads.fastq
Approx 15% complete for trimmed_reads.fastq
Approx 20% complete for trimmed_reads.fastq
Approx 25% complete for trimmed_reads.fastq
Approx 30% complete for trimmed_reads.fastq
Approx 35% complete for trimmed_reads.fastq
Approx 40% complete for trimmed_reads.fastq
Approx 45% complete for trimmed_reads.fastq
Approx 50% complete for trimmed_reads.fastq
Approx 55% complete for trimmed_reads.fastq
Approx 60% complete for trimmed_reads.fastq
Approx 65% complete for trimmed_reads.fastq
Approx 70% complete for trimmed_reads.fastq
Approx 75% complete for trimmed_reads.fastq
Approx 80% complete for trimmed_reads.fastq
Approx 85% complete for trimmed_reads.fastq
Approx 90% complete for trimmed_reads.fastq
Approx 95% complete for trimmed_reads.fastq
Analysis complete for trimmed_reads.fastq


# Коррекция ошибок (4 балла)

Получившиеся риды обработайте одним(любым) из инструментов, которые использованы и проанализированны в этой статье. Посчитайте TP, FP, TN, FN.

Запуск BFC с параметрами:

./bfc  -k 21 -t 4 trimmed_reads.fastq > corrected_output_trimmed.fasta

In [29]:
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

/usr/local/lib/python3.10/dist-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [41]:
def multiple_alignment(true_read, sequenced_read, corrected_read):
    alignments = pairwise2.align.globalxx(true_read, sequenced_read)
    seq_aligned, true_aligned = alignments[0][0], alignments[0][1]

    alignments = pairwise2.align.globalxx(true_read, corrected_read)
    corr_aligned, true_aligned = alignments[0][0], alignments[0][1]

    return seq_aligned, corr_aligned, true_aligned

def calculate_confusion_matrix(true_aligned, sequenced_aligned, corrected_aligned):
    TP, FP, TN, FN = 0, 0, 0, 0
    for true_base, sequenced_base, corrected_base in zip(true_aligned, sequenced_aligned, corrected_aligned):
        # Если в истинном риде и секвенированном риде символы совпадают
        if true_base == sequenced_base:
            # Если в исправленном риде символы совпадают
            if true_base == corrected_base:
                TP += 1  # True Positive
            else:
                FN += 1  # False Negative
        else:
            # Если в исправленном риде символы совпадают
            if true_base == corrected_base:
                TN += 1  # True Negative
            else:
                FP += 1  # False Positive

    return TP, FP, TN, FN

In [40]:
seq_reads = list(SeqIO.parse('sequenced_reads.fastq', 'fastq'))
true_seqes = {}
for i in range(len(seq_reads)):
  record = seq_reads[i]
  read_id = int(record.id[5:])
  start = error_info[read_id][0][0]
  length = len(record.seq)
  true_seqes[read_id] = sequence[start: start+length]

In [42]:
total_TP, total_FP, total_TN, total_FN = 0, 0, 0, 0

corrected_reads = list(SeqIO.parse('corrected_output_trimmed.fasta', 'fastq'))
trimmed_reads = list(SeqIO.parse('trimmed_reads.fastq', 'fastq'))

for i in range(len(trimmed_reads)):
    record = trimmed_reads[i]
    read_id = int(record.id[5:])
    sequenced_read = str(record.seq)

    corrected_read = str(corrected_reads[i].seq)

    true_read = true_seqes[read_id]

    seq_aligned, corr_aligned, true_aligned = multiple_alignment(true_read, sequenced_read, corrected_read)

    TP, FP, TN, FN = calculate_confusion_matrix(true_aligned, seq_aligned, corr_aligned)
    total_TP += TP
    total_FP += FP
    total_TN += TN
    total_FN += FN

# Теперь у вас есть общие значения метрик для всего набора данных
print("Total TP:", total_TP)
print("Total FP:", total_FP)
print("Total TN:", total_TN)
print("Total FN:", total_FN)

Total TP: 1951029
Total FP: 4606427
Total TN: 786343
Total FN: 8984


In [44]:
s = total_TP + total_FP + total_TN + total_FN
print("Total TP:", total_TP/s)
print("Total FP:", total_FP/s)
print("Total TN:", total_TN/s)
print("Total FN:", total_FN/s)

Total TP: 0.2653456521156683
Total FP: 0.6264875490001541
Total TN: 0.10694494859973427
Total FN: 0.001221850284443319


FP (False Positives - Ложноположительные):
Доля ложноположительных результатов составляет примерно 62.65%. Это означает, что около 62.65% исправлений, сделанных алгоритмом коррекции, были неправильными, т.е. исправления, которые не были необходимы и могли привести к искажению исходной информации.

TP (True Positives - Истинноположительные):
Доля истинноположительных результатов составляет примерно 26.53%. Это означает, что только около 26.53% всех ошибок в исходных ридах были правильно исправлены алгоритмом коррекции. Чем выше этот показатель, тем лучше исправления.

TN (True Negatives - Истинноотрицательные):
Доля истинноотрицательных результатов составляет примерно 10.69%. Это количество нуклеотидов, которые алгоритм коррекции правильно оставил без изменений. Это важно для сохранения правильности исходной информации.

FN (False Negatives - Ложноотрицательные):
Доля ложноотрицательных результатов невелика и составляет примерно 0.12%. Это количество нуклеотидов, которые были неправильно оставлены без изменений, хотя они требовали исправления.